In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

def plot_COVID1(region: str, *args):
    """
    args is a tuple of column names to be plotted
    e.g
    plot_COVID("Piemonte",'totale_casi','deceduti')
    will plot the overall daily results for 'totale_casi','deceduti' in Piemonte
    
    An additional column 'nuovi_casi' has been added.
    """

# The keys of the dictionary 'col' are the vailable columns (strings). The values are the colors for plotting 
    col={'ricoverati_con_sintomi':'C0',
         'terapia_intensiva':'C1',
         'totale_ospedalizzati':'C2',
         'isolamento_domiciliare':'C3',
         'totale_attualmente_positivi':'C4',
         'nuovi_attualmente_positivi':'C5',
         'dimessi_guariti':'C6',
         'deceduti':'C7',
         'totale_casi':'0',
         'tamponi':'C6',
         'nuovi_casi':'C8'}
    
# Load the data from the official repository. Notice that you need to download the file in 'raw' format non in html.
# The output is a pandas Dataframe. You can print it out.
    if region=='Italia':
        TotalByDate=pd.read_csv("https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv")
    else:
        TotalByDate=pd.read_csv("https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv")
        TotalByDate=TotalByDate.loc[lambda df: df['denominazione_regione']==region,:]
        # fix indices to be [0,1,.........,len(TotalByDate)-1]
        TotalByDate['entry']=list(range(len(TotalByDate)))
        TotalByDate.set_index('entry',inplace=True)

# Add column 'nuovi_casi'
    TotalByDate['nuovi_casi']=[0]+[TotalByDate['totale_casi'][i]-TotalByDate['totale_casi'][i-1] for i in range(1,len(TotalByDate))]

# Retrieve and format the x-axis labels
    xx=list(TotalByDate['data'])
    xx=[d[5:10].replace('02-','Feb ').replace('03-','Mar ')  for d in xx]

# Create the figure object
    fig1,ax1 = plt.subplots(figsize=(12,16))

    for label in ax1.xaxis.get_ticklabels():
        label.set_rotation(70)
        label.set_fontsize(14)

# Set and Format title
    title=region+" "+xx[0]+"-"+xx[-1]
            
    ax1.set_title(title,fontsize='28',pad=30)

# Set log scale on y-axis
    ax1.set_yscale('log')

# Set range on y-axis
# Define and format the y-axis labels
    if region in ['Italia','Lombardia']:
        ax1.set_ylim(5,50000)
        yticks = ([10,20,40,60,80]
              +[100,200,400,600,800]
              +[1000,2000,4000,6000,8000]
              +[10000,20000,40000,60000]
                 )
    else:
        ax1.set_ylim(1,10000)
        yticks = ([1,2,4,6,8]
              +[10,20,40,60,80]
              +[100,200,400,600,800]
              +[1000,2000,4000,6000,8000]
              +[10000]
                 )

    ax1.set_yticks(yticks)
    plt.yticks(fontsize='16')
# Display as integers    
    ax1.yaxis.set_major_formatter(FormatStrFormatter('%5d'))

# Set width of the curves
    lwidth=3
# Turn on grid
    ax1.grid(b=True,which='both')
    
# Loop on the required plots
    for name in args:
        if name == 'tamponi':
            TotalByDate[name]=TotalByDate[name]/4

        yy=list(TotalByDate[name])
        labelname=''
        if name == 'tamponi':
            labelname=name+'/4'
        else:
            labelname=name
            
        if name == 'nuovi_casi':
            ax1.plot(xx[1:],yy[1:],color=col[name],linewidth=lwidth,marker="o",markersize=8,label=labelname);
        else:
            ax1.plot(xx,yy,color=col[name],linewidth=lwidth,marker="o",markersize=8,label=labelname);
        
# Add legends using the 'label' of each curve
    ax1.legend(fontsize='14');
# Save the plots in (pdf and) jpg format
#    fig1.savefig(title.replace(' ','_',3)+".pdf")
    fig1.savefig(title.replace(' ','_',3)+"XX.jpg")

    return



In [16]:
#plot_COVID('Emilia Romagna','totale_casi','nuovi_casi','terapia_intensiva','deceduti','dimessi_guariti')

In [17]:
#plot_COVID_all('totale_casi','nuovi_casi','terapia_intensiva','deceduti','dimessi_guariti')

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

def plot_COVID(region: str, *args):
    """
    args is a tuple of column names to be plotted
    e.g
    plot_COVID("Piemonte",'totale_casi','deceduti')
    will plot the overall daily results for 'totale_casi','deceduti' in Piemonte
    
    Additional columns 'nuovi_casi', 'decessi_giorno', 'ricoveri_ICU_giorno' have been introduced.
    Daily data use dashed lines. Cumulative data use continuous lines.
    'Alto Adige' combines P.A. Bolzano and P.A. Trento.
    """

# The keys of the dictionary 'col' are the vailable columns (strings). The values are the colors for plotting 
    col={'ricoverati_con_sintomi':'C8',
         'nuovi_ricoverati_con_sintomi':'C8',
         'terapia_intensiva':'red',
         'nuovi_terapia_intensiva':'red',
         'totale_ospedalizzati':'C2',
         'isolamento_domiciliare':'C1',
         'totale_attualmente_positivi':'C4',
         'nuovi_attualmente_positivi':'C5',
         'dimessi_guariti':'green',
         'nuovi_dimessi_guariti':'green',
         'deceduti':'C0',
         'nuovi_deceduti':'C0',
         'tamponi':'C6',
         'nuovi_tamponi':'C6',
         'totale_casi':'0',
         'nuovi_casi':'0'}
    
# Load the data from the official repository. Notice that you need to download the file in 'raw' format non in html.
# The output is a pandas Dataframe.
    if region=='Italia':
        TotalByDate=pd.read_csv("https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv")
    else:
        TotalByDate=pd.read_csv("https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv")
        if region=='Alto Adige':
# Dataframe for 'Alto Adige' is the sum of 'P.A. Bolzano' and 'P.A. Trento'
            TotalByDate1=TotalByDate.loc[lambda df: df['denominazione_regione']=='P.A. Bolzano',:]
            TotalByDate2=TotalByDate.loc[lambda df: df['denominazione_regione']=='P.A. Trento',:]
            TotalByDate=TotalByDate1.add(TotalByDate1)
            TotalByDate['denominazione_regione']=['Alto Adige' for i in range(len(TotalByDate))]
            TotalByDate['data']=TotalByDate1['data']
        else:
            TotalByDate=TotalByDate.loc[lambda df: df['denominazione_regione']==region,:]
# fix indices to be [0,1,.........,len(TotalByDate)-1]
        TotalByDate['entry']=list(range(len(TotalByDate)))
        TotalByDate.set_index('entry',inplace=True)

# Add columns 'nuovi_casi', 'nuovi_decessi_giorno', 'nuovi_terapia_intensiva',
#             'nuovi_dimessi_guariti'
    daily=['nuovi_casi','nuovi_deceduti','nuovi_terapia_intensiva','nuovi_dimessi_guariti',
          'nuovi_tamponi','nuovi_ricoverati_con_sintomi']
    TotalByDate['nuovi_casi']=[0]+[TotalByDate['totale_casi'][i]-TotalByDate['totale_casi'][i-1] for i in range(1,len(TotalByDate))]
    TotalByDate['nuovi_deceduti']=[0]+[TotalByDate['deceduti'][i]-TotalByDate['deceduti'][i-1] for i in range(1,len(TotalByDate))]
    TotalByDate['nuovi_terapia_intensiva']=[0]+[TotalByDate['terapia_intensiva'][i]-TotalByDate['terapia_intensiva'][i-1] for i in range(1,len(TotalByDate))]
    TotalByDate['nuovi_dimessi_guariti']=[0]+[TotalByDate['dimessi_guariti'][i]-TotalByDate['dimessi_guariti'][i-1] for i in range(1,len(TotalByDate))]
    TotalByDate['nuovi_tamponi']=[0]+[TotalByDate['tamponi'][i]-TotalByDate['tamponi'][i-1] for i in range(1,len(TotalByDate))]
    TotalByDate['nuovi_ricoverati_con_sintomi']=[0]+[TotalByDate['ricoverati_con_sintomi'][i]
                                                     -TotalByDate['ricoverati_con_sintomi'][i-1] for i in range(1,len(TotalByDate))]

# Retrieve and format the x-axis labels
    xx=list(TotalByDate['data'])
    xx=[d[5:10].replace('02-','Feb ').replace('03-','Mar ')  for d in xx]

# Set width of the curves
    lwidth=3

# Create the figure object
    fig1,ax1 = plt.subplots(figsize=(12,16))

# Setup figure parameters
    for label in ax1.xaxis.get_ticklabels():
        label.set_rotation(70)
        label.set_fontsize(14)

# Set log scale on y-axis
    ax1.set_yscale('log')

# Set range on y-axis
# Define and format the y-axis labels
    if region in ['Italia','Lombardia']:
        ax1.set_ylim(5,50000)
        yticks = ([10,20,40,60,80]
              +[100,200,400,600,800]
              +[1000,2000,4000,6000,8000]
              +[10000,20000,40000,60000,80000]
              +[100000,200000]
                 )
        ax1.set_yticks(yticks)
        plt.yticks(fontsize='16')
    else:
        ax1.set_ylim(1,10000)
        yticks = ([1,2,4,6,8]
              +[10,20,40,60,80]
              +[100,200,400,600,800]
              +[1000,2000,4000,6000,8000]
              +[10000,20000,40000]
                 )
        ax1.set_yticks(yticks)
        plt.yticks(fontsize='16')


# Display as integers    
    ax1.yaxis.set_major_formatter(FormatStrFormatter('%5d'))

# Turn on grid
    ax1.grid(b=True,which='both')

# Loop on the required plots
    for name in args:

        if name == 'tamponi':
            TotalByDate[name]=TotalByDate[name]/4

        yy=list(TotalByDate[name])

        labelname=''
        if name == 'tamponi':
            labelname=name+'/4'
        else:
            labelname=name

        yy=list(TotalByDate[name])

        if name in daily:
            linsty='dashed'
#            ax1.plot(xx[1:],yy[1:],color=col[name],linewidth=lwidth,linestyle='dashed',
#                     marker="o",markersize=8,label=name);
        else:
            linsty='solid'

        ax1.plot(xx,yy,color=col[name],linewidth=lwidth,linestyle=linsty,
                     marker="o",markersize=8,label=labelname);
        
# Add legends using the 'label' of each curve
    ax1.legend(fontsize='14',handlelength=5);

# Set and Format title
    title=region+" "+xx[0]+"-"+xx[-1]
    ax1.set_title(title,fontsize='28',pad=30)

# Save
    counts=title.count(' ')
    fig1.savefig(title.replace(' ','_',counts)+".jpg")
# Close
    plt.close()

    return
    
    
def plot_COVID_all(*args):
    regions=['Italia','Abruzzo','Alto Adige','Basilicata','Calabria','Campania','Emilia Romagna','Friuli Venezia Giulia',
            'Lazio','Liguria','Lombardia','Marche','Molise','Piemonte','Puglia','Sardegna','Sicilia',
            'Toscana','Umbria','Valle d\'Aosta','Veneto']
    
    for region in regions:
        print('region:',region)
        plot_COVID(region,*args)
    
    return
 

In [5]:
#plot_COVID('Lombardia','nuovi_casi', 'nuovi_terapia_intensiva', 'nuovi_deceduti')

#plot_COVID('Alto Adige','nuovi_casi_giorno', 'decessi_giorno', 'ricoveri_ICU_giorno','terapia_intensiva')

plot_COVID_all('totale_casi','nuovi_casi','terapia_intensiva','nuovi_terapia_intensiva',
               'deceduti','nuovi_deceduti','tamponi')

#plot_COVID('Piemonte','totale_casi','nuovi_casi','terapia_intensiva','nuovi_terapia_intensiva',
#               'deceduti','nuovi_deceduti','tamponi')

#plot_COVID('Italia','nuovi_casi','nuovi_terapia_intensiva',
#           'nuovi_ricoverati_con_sintomi','nuovi_tamponi')

#plot_COVID('Valle d\'Aosta','totale_casi','nuovi_casi','terapia_intensiva','nuovi_terapia_intensiva',
#               'deceduti','nuovi_deceduti')

region: Italia
region: Abruzzo
region: Alto Adige
region: Basilicata
region: Calabria
region: Campania
region: Emilia Romagna
region: Friuli Venezia Giulia
region: Lazio
region: Liguria
region: Lombardia
region: Marche
region: Molise
region: Piemonte
region: Puglia
region: Sardegna
region: Sicilia
region: Toscana
region: Umbria
region: Valle d'Aosta
region: Veneto
